In [ ]:
#hide
from doc_collection.core import *

# doc_collection

> A tool for collecting documentation of python libs.

## Install

`pip install doc_collection`

## How to use

Firstly, you need to pip list of packages to be able to collect data from it:

In [ ]:
doc_collection.pip_top_hundred()

2

Now you are ready to extract data from all libs you've pipped and it's dependencies:

In [ ]:
doc_collection.extract()

Command above will return DataFrame of three columns: __text__ contains documentation of an object, __paths__ contains all names of an object (can be more than one) and __library__ contains library of an object.